In [12]:
import torch
import torch.nn as nn# loos ftn, conv ,linear
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader
import torchvision.datasets as datasets
import torchvision.transforms as transforms


In [13]:
class NN(nn.Module):
    def __init__(self, input_size,num_classes):
        super(NN,self).__init__()
        self.l1=nn.Linear(input_size, 50)
        self.l2= nn.Linear(50,num_classes)

    def forward(self, x):
        x=F.relu(self.l1(x))# use F if we want to call activation ftn in forward instead of constructor
        x=self.l2(x)
        return x



In [14]:
# Device configuration
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [15]:
input_size= 784
num_classes=10
learning_rate= 0.001
batch_size= 64
num_epochs=2

In [16]:
# MNIST dataset 
train_dataset = datasets.MNIST(root='./data', 
                                           train=True, 
                                           transform=transforms.ToTensor(),  
                                           download=True)

test_dataset = datasets.MNIST(root='./data', 
                                          train=False, 
                                          transform=transforms.ToTensor())

# Data loader
train_loader = DataLoader(dataset=train_dataset, 
                                           batch_size=batch_size, 
                                           shuffle=True)

test_loader = DataLoader(dataset=test_dataset, 
                                          batch_size=batch_size, 
                                          shuffle=False)

In [17]:
model=NN(input_size=input_size, num_classes=num_classes).to(device)

In [18]:
criterion = nn.CrossEntropyLoss()
optimizer= optim.Adam(model.parameters(),lr=learning_rate)

In [19]:
for epoch in range (num_epochs):
    for i,(data, targets) in enumerate(train_loader):
        data=data.to(device)
        targets=targets.to(device)

        data=data.reshape(-1, 28*28)
        pred=model(data)
        loss =criterion(pred, targets)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()


In [21]:
def check_accuracy(loader,model):
    num_correct=0
    num_samples=0
    model.eval()

    with torch.no_grad():
        for x,y in loader:
            x=x.to(device)
            y=y.to(device)
            x=x.reshape(-1, 28*28)

            pred= model(x)
            _, predicted = pred.max(1)
            num_correct += (predicted==y).sum()
            num_samples += predicted.size(0)

        acc = 100.0 * num_correct / num_samples
    print(f'Accuracy of the network on the 10000 test images: {acc} %')
    model.train()
    


In [23]:
check_accuracy(test_loader, model)

Accuracy of the network on the 10000 test images: 94.72999572753906 %


tensor(94.7300, device='cuda:0')